In [ ]:
import random 
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pandas as pd 
import matplotlib.pyplot as plt

# 1.1 Secretary: Stationary case

## 1.1.1 generate offline data, online data 

In [ ]:
# set parameter 
gamma = 200
T = 1000

# offline data
randseq = np.random.randint(1, gamma+1, (1,T))/gamma 

# online data(multiple scenario)
scenario=100
replicate=1000
scenarioList=np.random.randint(1, gamma+1, size=(scenario,gamma))/gamma

In [ ]:
# function for storing data
def store(data,filename,index_name,column_name):
    df=pd.DataFrame(data,index=index_name,columns=column_name)
    df.to_csv(filename)

In [ ]:
# store data
store(randseq,'secretary/stationary/sec_stationary_offline_data.csv',['score'],[i for i in range(T)])
store(scenarioList,'secretary/stationary/sec_stationary_scenario_data.csv',['scenario%d'%i for i in range(scenario)],[i for i in range(gamma)])

## 1.1.2 O2O algorithm

In [ ]:
# function for implementing O2O
def secretary(gamma,T,z,randseq,optimize,station,scenarioList):
    theta = []
    theta.append(np.array([0,0]))
    grad = np.array([0,0])
    # compute lower bound of phi
    delta = 0.01 # probability parameter. lower delta means higher probability to hold but larger error gap
    L = 2
    K=2
    philb = z-2*K*L*np.sqrt(2*np.log(2/delta)/T) 
    xt = []
    for i in range(T):
        x = 0 if theta[i][0]*randseq[0,i]+theta[i][1] >= 0 else 1 
        xt.append(x)
        f = np.array([x*randseq[0,i], x]) 
        # compute gradient
        model = gp.Model('gradmodel')
        w = model.addVars(2, ub=1.0, vtype=GRB.CONTINUOUS,name='w')
        y = model.addVars(2, ub=1.0, vtype=GRB.CONTINUOUS,name='y')
        model.setObjective(theta[i][0]*w[0]+theta[i][1]*w[1]-(w[0]-y[0])**2-(w[1]-y[1])**2, GRB.MAXIMIZE)
        # use y to represent set S
        model.addConstr(y[0]>=philb)
        model.addConstr(y[1]==1/gamma)
        model.optimize()
        var = model.getVars()
        if optimize=='gradient':
            grad = np.array([var[0].x, var[1].x])-np.array([x*randseq[0,i], x])        
            ita=L/(pow(8*K*(i+1),0.5))
            new_theta = theta[i]-ita*grad
            theta.append(new_theta)
        else:
            grad = grad + np.array([var[0].x, var[1].x])-np.array([x*randseq[0,i], x])
            new_theta = -L*grad/max(np.sqrt(8*K*T),np.linalg.norm(grad))
            theta.append(new_theta)
    store(theta[1:],'secretary/%s/%s/sec_%s_%s_theta.csv'%(station,optimize,station,optimize),[i for i in range(T)],['theta0','theta1'])
    # online replicate
    scenario=len(scenarioList)
    resultList=[[] for i in range(scenario)]
    for i in range(scenario): 
        realseq=scenarioList[i]
        for j in range(replicate): 
            # Online algorithm
            index = np.random.randint(T, size = gamma) 
            counted=False
            for k in range(gamma): 
                theta0 = theta[index[k]]
                if theta0[0]*realseq[k]+theta0[1]<0:
                    resultList[i].append(realseq[k])
                    counted=True
                    break
            if not counted:
                resultList[i].append(realseq[-1])
        store(resultList,'secretary/%s/%s/sec_%s_%s_result.csv'%(station,optimize,station,optimize),['scenario'+str(i) for i in range(scenario)],[i for i in range(replicate)])

    # bid price for decision
    bid=[]
    for i in theta[1:]:
        bid.append(i[1]/(-i[0]))
    bid=sorted(bid,reverse=True)
    store(bid,'secretary/%s/%s/sec_%s_%s_bid.csv'%(station,optimize,station,optimize),[i for i in range(len(bid))],['bid price'])
    
    # hindsight
    hindsight=[]
    for i in range(scenario):
        hindsight.append(max(scenarioList[i]))

    # 1/e policy
    point=int(gamma/np.e)+1
    e_policy=[]
    for i in range(scenario):
        realseq=scenarioList[i]
        pre_max=max(realseq[:point])
        counted=False
        for j in range(point,gamma):
            if realseq[j]>pre_max:
                e_policy.append(realseq[j])
                counted=True
                break
        if not counted:
            e_policy.append(realseq[-1])

    # O2O
    percent25=[]
    percent75=[]
    mean=[]
    for i in range(scenario):
        result=resultList[i]
        percent25.append(np.percentile(result, 25))
        percent75.append(np.percentile(result, 75))
        mean.append(np.mean(result))

    zipped = zip(e_policy,hindsight,percent25,percent75,mean)
    sort_zipped=sorted(zipped,key=lambda x:x[0],reverse=True)
    zip(*sort_zipped)
    totalList=np.array(sort_zipped)
    
    # regret
    regret=totalList[:,1]-totalList[:,4]
    store(regret,'secretary/%s/%s/sec_%s_%s_regret.csv'%(station,optimize,station,optimize),[i for i in range(scenario)],['regret'])
    xaxis=range(1,scenario+1)
    fig, ax = plt.subplots() 
    ax.plot(xaxis, regret, 'r',label='Average regret') 
    ax.set_xlabel('scenario') 
    ax.set_ylabel('Regret Value')
    ax.set_ylim(0,1.2)
    ax.set_title('Regret of %s secretary problem under online %s descent'%(station,optimize)) 
    ax.legend() 
    fig.savefig('secretary/%s/%s/sec_%s_%s_regret'%(station,optimize,station,optimize) )

    xaxis=range(1,scenario+1)
    fig, ax = plt.subplots() 
    ax.plot(xaxis, totalList[:,1], 'r*-',label='Hindsight') 
    ax.plot(xaxis, totalList[:,0], 'b-',label='1/e Law Policy') 
    ax.plot(xaxis, totalList[:,2], 'g--',label='25-th quantile') 
    ax.plot(xaxis, totalList[:,4], 'k-',label='mean')
    ax.plot(xaxis, totalList[:,3], 'g:',label='75-th quantile') 
    ax.set_xlabel('scenario') 
    ax.set_ylabel('Candidate Value')
    ax.set_ylim(0,1.2)
    ax.set_title('%s secretary problem by online %s descent'%(station,optimize)) 
    ax.legend() 
    fig.savefig('secretary/%s/%s/sec_%s_%s_result'%(station,optimize,station,optimize))

In [ ]:
# train theta offline

# get upperbound using SAA, thus get programming only with constarint
ub = []
for i in range(T):
    sampleseq = np.random.randint(1, gamma+1, gamma)/gamma
    ub.append(max(sampleseq))
z = sum(ub)/T
store(z,'secretary/stationary/sec_stationary_z.csv',['z'],['value'])

In [ ]:
# train by online gradient descent
secretary(gamma,T,z,randseq,'gradient','stationary',scenarioList)

In [ ]:
# train by online mirror descent
secretary(gamma,T,z,randseq,'mirror','stationary',scenarioList)

# 1.2 Secretary: Non-stationary case

## 1.1.1 generate offline data, online data

In [ ]:
def sec_nonstation_gen(gamma):
    sampleseq=[]
    for j in range(gamma):
        rand_value=np.random.rand()
        if rand_value>0.5:
            sampleseq.append((gamma-j)/gamma)
        else:
            sampleseq.append(0)
    return sampleseq

In [ ]:
def sec_singleGen(j,gamma):
    rand_value=np.random.rand()
    if rand_value>0.5:
        return (gamma-j)/gamma
    else:
        return 0

In [ ]:
# set parameter 
gamma = 200
T = 1000

# offline data
randseq = []
for i in range(T):
    ind=np.random.randint(gamma)
    randseq.append(sec_singleGen(ind,gamma))
randseq=np.array(randseq).reshape(1,T)
store(randseq,'secretary/nonstationary/sec_nonstationary_offline_data.csv',['score'],[i for i in range(T)])

# online data(multiple scenario)
scenario=100
replicate=1000
scenarioList=[]
for i in range(scenario):
    scenarioList.append(sec_nonstation_gen(gamma))
store(scenarioList,'secretary/nonstationary/sec_nonstationary_scenario_data.csv',['scenario%d'%i for i in range(scenario)],[i for i in range(gamma)])

# upper bound
ub = []
for i in range(T):
    sampleseq=sec_nonstation_gen(gamma)
    ub.append(max(sampleseq))
z = sum(ub)/T 
store(z,'secretary/nonstationary/sec_nonstationary_z.csv',['z'],['value'])

## 1.1.2 O2O algorithm

In [ ]:
# train by online gradient descent
secretary(gamma,T,z,randseq,'gradient','nonstationary',scenarioList)

In [ ]:
# train by online mirror descent
secretary(gamma,T,z,randseq,'mirror','nonstationary',scenarioList)

# 2.1 Resource allocation:case 1

## 2.1.1 generate online data, offline data

In [ ]:
# set parameter
T = 10000
gamma = 2000
lamb = np.array([[9, 18, 27], [3, 6, 9]])
miu = lamb[0]/2+lamb[1]/2
beta = np.array([0.85, 0.9, 0.95])
product = np.multiply(miu, beta)
cstar=50

# offline data
off_demand=[]
for i in range(T):
    # choose period
    index = np.random.randint(0,2)
    samplamd = lamb[index]
    off_demand.append(np.random.poisson(lam = samplamd, size = 3))
off_demand=np.array(off_demand).T
store(off_demand,'resource/case1/case1_offline.csv',['customer'+str(i) for i in range(3)],[i for i in range(T)])

# online data
online_demand=[]
for i in range(gamma):
    index = np.random.randint(0,2)
    index = 0 if i <=gamma/2 else 1
    reallamb = lamb[index]
    online_demand.append(np.random.poisson(lam = reallamb, size = 3))
online_demand=np.array(online_demand).T
store(online_demand,'resource/case1/case1_online.csv',['customer'+str(i) for i in range(3)],[i for i in range(gamma)])

## 2.1.2 O2O algorithm

In [ ]:
def resource(off_demand,online_demand,T,gamma,beta,product,cstar,case,optimize):
    # offline
    L=2
    K=3
    delta = 0.01 
    theta = []
    theta.append(np.array([1,1,1]))
    grad = np.array([0,0,0])
    for i in range(T):
        demand = off_demand[:,i]
        # solve the CPS
        model = gp.Model('gradmodel')
        x = model.addVars(3, ub=demand, vtype=GRB.CONTINUOUS,name='x')
        model.setObjective(-np.dot(theta[i], product)+(theta[i][0]*(x[0])+theta[i][1]*(x[1])+theta[i][2]*(x[2])), GRB.MAXIMIZE)
        model.addConstr(x[0]+x[1]+x[2]<=cstar)
        model.optimize()
        var = model.getVars()
        # compute gradient
        if optimize=='gradient':
            grad=np.array([var[0].x, var[1].x, var[2].x])-product
            ita=L/(pow(8*K*(i+1),0.5))
            new_theta=theta[i]-ita*grad
            theta.append(new_theta)
        else:
            grad = grad + np.array([var[0].x, var[1].x, var[2].x])-product
            delta = -1/max(np.sqrt(8*K*T), np.linalg.norm(grad))
            theta.append(delta*grad)
    store(theta[1:],'resource/%s/%s/res_%s_%s_theta.csv'%(case,optimize,case,optimize),[i for i in range(T)],['customer'+str(i) for i in range(K)])
    
    # priority list
    priority=[]
    for weight in theta:
        prior_oder = np.argsort(-weight)
        priority.append(prior_oder)
    store(priority[1:],'resource/%s/%s/res_%s_%s_prio.csv'%(case,optimize,case,optimize),[i for i in range(T)],['order'+str(i) for i in range(K)])
    
    
    # online 
    xt = []
    rate = []
    indexseq = np.random.randint(gamma, size = gamma)
    demand=[]
    for i in range(gamma):
        thetat = theta[indexseq[i]]
        realdemand = online_demand[:,i]
        demand.append(realdemand)
        model = gp.Model('onlinemodel')
        x = model.addVars(3, ub=realdemand, vtype=GRB.CONTINUOUS,name='x')
        model.setObjective(-np.dot(thetat, product)+(thetat[0]*(x[0])+thetat[1]*(x[1])+thetat[2]*(x[2])), GRB.MAXIMIZE)
        model.addConstr(x[0]+x[1]+x[2]<=cstar)
        model.optimize()
        xt.append(np.array([x[0].x, x[1].x, x[2].x]))    
        rate.append(sum(xt)/sum(demand))
    shortage=[]
    for i in range(gamma):
        shortage.append(np.maximum(0,beta-rate[i]))
    store(xt,'resource/%s/%s/res_%s_%s_result.csv'%(case,optimize,case,optimize),[i for i in range(gamma)],['customer'+str(i) for i in range(K)])
    store(rate,'resource/%s/%s/res_%s_%s_rate.csv'%(case,optimize,case,optimize),[i for i in range(gamma)],['customer'+str(i) for i in range(K)])
    store(shortage,'resource/%s/%s/res_%s_%s_shortage.csv'%(case,optimize,case,optimize),[i for i in range(gamma)],['customer'+str(i) for i in range(K)])
    
    # plot
    xaxis=range(gamma)
    rate=np.array(rate)
    fig,ax=plt.subplots()
    ax.plot(xaxis, rate[:,0], 'r-',label='customer1,beta=%s'%beta[0]) 
    ax.plot(xaxis, rate[:,1], 'b:',label='customer2,beta=%s'%beta[1]) 
    ax.plot(xaxis, rate[:,2], 'k--',label='customer3,beta=%s'%beta[2]) 
    ax.plot(gamma-1,beta[0],'ro')
    ax.plot(gamma-1,beta[1],'bo')
    ax.plot(gamma-1,beta[2],'ko')
    ax.set_xlabel('period') 
    ax.set_ylabel('Attained fill rate')
    ax.set_ylim(0,1.1)
    ax.set_title('Attained fill rate of %s under online %s descent'%(case,optimize)) 
    ax.legend() 
    fig.savefig('resource/%s/%s/rate_%s_%s'%(case,optimize,case,optimize))
    
    shortage=np.array(shortage)
    fig,ax=plt.subplots()
    ax.plot(xaxis, shortage[:,0], 'r-',label='customer1,beta=%s'%beta[0]) 
    ax.plot(xaxis, shortage[:,1], 'b:',label='customer2,beta=%s'%beta[1]) 
    ax.plot(xaxis, shortage[:,2], 'k--',label='customer3,beta=%s'%beta[2]) 
    ax.set_xlabel('period') 
    ax.set_ylabel('Fill rate gap')
    ax.set_ylim(0,1.1)
    ax.set_title('Fill rate gap of %s under online %s descent'%(case,optimize)) 
    ax.legend() 
    fig.savefig('resource/%s/%s/shortage_%s_%s'%(case,optimize,case,optimize))

In [ ]:
# online gradient descent
resource(off_demand,online_demand,T,gamma,beta,product,cstar,'case1','gradient')

In [ ]:
# online mirror descent
resource(off_demand,online_demand,T,gamma,beta,product,cstar,'case1','mirror')

# 2.2 Resource allocation:case 2

## 2.2.1 generate data

In [ ]:
# set parameter
T = 10000
gamma = 2000
lamb = np.array([[3, 6, 9],[9, 18, 27]])
miu = lamb[0]/2+lamb[1]/2
beta = np.array([0.85, 0.9, 0.95])
product = np.multiply(miu, beta)
cstar=50

# offline data
off_demand=[]
for i in range(T):
    # choose period
    index = np.random.randint(0,2)
    samplamd = lamb[index]
    off_demand.append(np.random.poisson(lam = samplamd, size = 3))
off_demand=np.array(off_demand).T
store(off_demand,'resource/case2/case2_offline.csv',['customer'+str(i) for i in range(3)],[i for i in range(T)])

# online data
online_demand=[]
for i in range(gamma):
    index = np.random.randint(0,2)
    index = 0 if i <=gamma/2 else 1
    reallamb = lamb[index]
    online_demand.append(np.random.poisson(lam = reallamb, size = 3))
online_demand=np.array(online_demand).T
store(online_demand,'resource/case2/case2_online.csv',['customer'+str(i) for i in range(3)],[i for i in range(gamma)])

## 2.2.2 O2O algorithm

In [ ]:
# online gradient descent
resource(off_demand,online_demand,T,gamma,beta,product,cstar,'case2','gradient')

In [ ]:
# online mirror descent
resource(off_demand,online_demand,T,gamma,beta,product,cstar,'case2','mirror')